In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()
api_key_1 = os.getenv('API_KEY_1') #Gjør dette med enn .env fil
print(api_key_1)
base_URL = os.getenv('DATABASE_URL')



if not api_key_1 or not base_URL:
    raise ValueError ("DATABASE/APIKEY mangler i .env filen")

city = "Trondheim"
start_date = datetime(2025, 1, 16)
end_date = datetime(2025, 3, 16)

vaerdata = []

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    url = f"{base_URL}/history.json?key={api_key_1}&q={city}&dt={date_str}"



    response = requests.get(url)

    if response.status_code == 200: #Kontroll for vellykket API-kall
        data = response.json()
    else: 
        print(f"Feil ved henting av data fra API-et for {date_str} : {response.status_code} ")
        current_date += timedelta(days=1)
        continue


    for hour_data in data.get("forecast", {}).get("forecastday", [])[0].get("hour", []):
        tidspunkt = hour_data.get("time", "Ukjent tidspunk")
        temperatur = hour_data.get("temp_c", "Ukjent temperatur")
        vindhastighet = hour_data.get("wind_kph", "Ukjent vindhastighet")
        pressure_mb = hour_data.get("pressure_mb", "Ukjent trykk" )
        nedbør = hour_data.get("precip_mm", "Ukjent nedbør")

        vaerdata.append({
            "Tidspunkt" : tidspunkt, 
            "Temperatur (°C)" : temperatur, 
            "Vindhastighet (km/t)" : vindhastighet,
            "Trykk" : pressure_mb, 
            "Nedbør (mm)" : nedbør
        })

    current_date += timedelta (days=1)

df = pd.DataFrame(vaerdata)

df.to_csv("trondheim_vaerdata_full_1.csv", index = False, encoding="utf-8")

print("data lagret i trondheim_vaerdata_full_1.csv")

print(df.head())

print("hallo")





b2fc4b14d4e74d6cb31113130252003
data lagret i trondheim_vaerdata_full_1.csv
          Tidspunkt  Temperatur (°C)  Vindhastighet (km/t)   Trykk  \
0  2025-01-16 00:00              5.7                  17.5  1014.0   
1  2025-01-16 01:00              6.9                  21.9  1014.0   
2  2025-01-16 02:00              6.8                  19.8  1015.0   
3  2025-01-16 03:00              6.4                  22.0  1015.0   
4  2025-01-16 04:00              6.7                  20.9  1015.0   

   Nedbør (mm)  
0          2.1  
1          0.9  
2          0.3  
3          0.1  
4          0.0  
hallo
